# Investigate HIV Datasets

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

The human immunodeficiency virus (HIV) is a lentivirus (a subgroup of retrovirus) that causes HIV infection and over time acquired immunodeficiency syndrome (AIDS).AIDS is a condition in humans in which progressive failure of the immune system allows life-threatening opportunistic infections and cancers to thrive. Without treatment, average survival time after infection with HIV is estimated to be 9 to 11 years, depending on the HIV subtype. ([Source](https://en.wikipedia.org/wiki/HIV)) It is one of the most dangerous virus that is threatening human beings' lives, not only adults but also children. 

In this report I will investigate a few HIV datasets which are downloaded from [Gapmind](http://www.gapminder.org/data/). The datasets are listed below:

* **Child-HIV .csv:** Child death rate in year 2008 in different countries, per thousand
* **1-59-HIV .csv:** Total number of child death in year 2008 in different countries
* **indicator_hiv_plwha.csv:** Total number of HIV livings in different years and countries 
* **Indicator_Estimated new HIV cases All ages.csv:** Total number of people among all age groups that were newly infected with HIV during different years
* **Indicator_HIV Incidence Ages 15-49.csv:** Number of people between 15 and 49 years old that were newly infected with HIV during the given year per 100 population
* **Indicator_Annual number of AIDS deaths.csv:** Total number of deaths due to AIDS during different years
* **indicator hiv estimated prevalence% 15-49.csv:** Number of people living with HIV per 100 population of age group 15-49


In order to investigate continent-wise distribution, I need convert these countries' name into continents', so an additinal dataset is imported during this clean process, which is downloaded from [this link](https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv).

I am hoping to answer a few questions from data in hands: 

* Which country is the most dangerous in terms of the number of people infected and death. 
* How these countrie

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.rcParams["figure.figsize"] = (12, 5) # set figure size

<a id='wrangling'></a>
## Data Wrangling

Information is stored in different dataset, so first I need to combine them and wrangle for future exploration. 

### General Properties

In [ ]:
# Load different datasets and combine and wrangle data for further investigation
child_death_per_k = pd.read_csv('dataset/Child-HIV .csv')
child_death_ttl = pd.read_csv('dataset/1-59-HIV .csv') 
conti_country = pd.read_csv('dataset/all.csv')
conti_country = conti_country.loc[:, ["name", "region"]]
conti_country.columns = ["Country", "Continent"]

child_death = pd.merge(child_death_ttl, child_death_per_k, how="left", on = "Gapminder name")
child_death.columns = ["Country", "Total", "Num_per_thousand"]
child_death = pd.merge(child_death, conti_country, how="left", on = "Country")

child_death.head()

In [ ]:
child_death.info()

In [ ]:
child_death.describe()

The new dataset `child_death` combines `Child-HIV .csv` and `1-59-HIV .csv`, which demonstrates total number of death and number per thousand in year 2008.

From above we can get some initial information:

* There are 178 observations in the dataset
* The average total number of death is 1130 but median is 6 (1.72 and 0.05 respectively for number per thousand), which means these two variables are highly biased
* There is a few missing data under continent column

In [ ]:
def melt_func(x):
    """
    Melt dataframe with similar data structure
    """
    idx = x.columns[0]
    return x.melt(id_vars = idx)

In [ ]:
def outer_merge(x, y):
    """
    Outer merge two dataframes with keys of country and year
    """
    return pd.merge(x, y, how="outer", on=["Country", "Year"])

In [ ]:
ttl_living_HIV = pd.read_csv('dataset/indicator_hiv_plwha.csv')
ttl_living_HIV = melt_func(ttl_living_HIV)
ttl_living_HIV.columns = ['Country', 'Year', 'Num_HIV_living']

new_HIV_all_age = pd.read_csv('dataset/Indicator_Estimated new HIV cases All ages.csv')
new_HIV_all_age = melt_func(new_HIV_all_age)
new_HIV_all_age.columns = ['Country', 'Year', 'New_added_HIV_all_age']

new_HIV_15_49_pct = pd.read_csv('dataset/Indicator_HIV Incidence Ages 15-49.csv')
new_HIV_15_49_pct = melt_func(new_HIV_15_49_pct)
new_HIV_15_49_pct.columns = ['Country', 'Year', 'New_added_HIV_15-49_pct']

ann_death = pd.read_csv('dataset/Indicator_Annual number of AIDS deaths.csv')
ann_death = melt_func(ann_death)
ann_death.columns = ['Country', 'Year', 'Annual_death']

est_pct_15_49 = pd.read_csv('dataset/indicator hiv estimated prevalence% 15-49.csv')
est_pct_15_49 = melt_func(est_pct_15_49)
est_pct_15_49.columns = ['Country', 'Year', 'Est_HIV_pct_15-49']

ttl_year_data = outer_merge(ttl_living_HIV, 
                    outer_merge(new_HIV_all_age,
                    outer_merge(new_HIV_15_49_pct, 
                    outer_merge(est_pct_15_49, ann_death))))

ttl_year_data = pd.merge(ttl_year_data, conti_country, how="left", on="Country")

ttl_year_data.head(10)

In [ ]:
ttl_year_data.info()

In [ ]:
ttl_year_data.describe()

The new dataset `ttl_year_data` combines all the rest datsets. Every dataset needs to be unpivoted first and then joined according to column `country` and `year`. 

From above we can also get some first information:

* There are total 9097 observations and 5 columns
* There are many missing data, including continents
* The largest number of HIV living is 5.6 * 10^6, which is extremely huge. The mean is 1.57 * 10^5 and the median is 1.2 * 10^4
* The largest number of new added HIV of all ages is 7.2 * 10^5. The mean is 3.5 * 10^4 and the median is 6.2 * 10^3.
* The mean of numbers of HIV newly infected from 15 to 49 is 0.46 and median is 0.42. This is pretty close. 
* The mean of numbers of HIV infected cases per hundred is 1.74 and median is 0.3. 
* The mean of annual death is 9050 and median is 350, which is highly skewed. 


### Data Cleaning

As previous initial exploration, we can see there are two kinds of missing data: numbers under different indicator column, strings udner continent column. Let's discuss them separately.

#### Missing data under different indicator columns

In [ ]:
cleaned_year_data = ttl_year_data.copy()
cleaned_year_data.groupby('Year').describe()

As I find previously there are a lot of missing data. Some common approaches to fill these are:

* filled with mean or median
* estimate with some algorithms
* look for new data source to fill the gap
* discard observations with missing data

Since values of different indicator are quite different among different years and countries, it is not accurate to use mean or median to fill in a specific year or country, so it wouldn't be a good idea to go with option one. It is also kind of time consuming to go with option two and three. So for simplicity, I will just subset data after 1990 and remain left missing data there.

In [ ]:
cleaned_year_data = cleaned_year_data.loc[cleaned_year_data.Year >= '1990',]

#### Missing data under continent column

Missing data under continent column is kind of easier to deal with. First I will query those of missing continent in two combined datasets, and then fill them in one by one.

In [ ]:
missing_conti = list(child_death.loc[pd.isnull(child_death.Continent), :].Country)
missing_conti

In [ ]:
cleaned_year_data.loc[pd.isnull(cleaned_year_data.Continent), :].Country

It's surprising to see some of countries' name are missing, so it's impossible to know which continent they belong. These observations will also be deleted. 

In [ ]:
cleaned_year_data = cleaned_year_data.loc[-pd.isnull(cleaned_year_data.Country), :]
missing_conti = list(cleaned_year_data.loc[pd.isnull(cleaned_year_data.Continent), :].Country)
missing_conti_set = set(missing_conti)
missing_conti_set

Of all these countries, they may have been reconstructed or reorganized. But in order to conduct further analysis, I will mannually set up a dictionary to map countries to continents.

In [ ]:
missing_conti_map = {
 'Abkhazia': 'Asia',
 'Akrotiri and Dhekelia': 'Europe',
 'Antarctica': 'Antarctica',
 'Bolivia': 'America',
 'Bonaire': 'America',
 'British Virgin Islands': 'America',
 'Brunei': 'Asia',
 'Cape Verde': 'Africa',
 'Channel Islands': 'Europe',
 'Chinese Taipei': 'Asia',
 'Christian': 'Africa',
 'Christmas Island': 'Asia',
 'Coastline': 'Nan',
 'Cocos Island': 'Asia',
 'Congo, Dem. Rep.': 'Africa',
 'Congo, Rep.': 'Africa',
 'Cook Is': 'Nan',
 "Cote d'Ivoire": 'Africa',
 'Cura\x8dao': 'Nan',
 'Czechoslovakia': 'Europe',
 'East Germany': 'Europe',
 'Eritrea and Ethiopia': 'Africa',
 'Faeroe Islands': 'Europe',
 'Falkland Is (Malvinas)': 'America',
 'Hawaiian Trade Zone': 'America',
 'Hong Kong, China': 'Asia',
 'Iran': 'Asia',
 'Kosovo': 'Europe',
 'Kyrgyz Republic': 'Asia',
 'Lao': 'Asia',
 'Macao, China': 'Asia',
 'Macedonia, FYR': 'Europe',
 'Micronesia, Fed. Sts.': 'Oceania',
 'Moldova': 'Europe',
 'Netherlands Antilles': 'Europe',
 'Ngorno-Karabakh': 'Europe',
 'North Korea': 'Asia',
 'North Yemen (former)': 'Asia',
 'Northern Cyprus': 'Europe',
 'Reunion': 'Africa',
 'Russia': 'Europe',
 'Saba': 'Europe',
 'Saint Eustatius': 'Europe',
 'Sark': 'Europe',
 'Serbia and Montenegro': 'Europe',
 'Serbia excluding Kosovo': 'Europe',
 'Slovak Republic': 'Europe',
 'Somaliland': 'Africa',
 'South Korea': 'Asia',
 'South Ossetia': 'Europe',
 'South Yemen (former)': 'Asia',
 'St. BarthŽlemy': 'Nan',
 'St. Helena': 'Africa',
 'St. Kitts and Nevis': 'America',
 'St. Lucia': 'America',
 'St. Martin': 'America',
 'St. Martin (French part)': 'Europe',
 'St. Vincent and the Grenadines': 'America',
 'St.-Pierre-et-Miquelon': 'America',
 'Svalbard': 'Europe',
 'Syria': 'Asia',
 'Taiwan': 'Asia',
 'Tanzania': 'Africa',
 'Transnistria': 'Europe',
 'U.S. Pacific Islands': 'America',
 'USSR': 'Asia',
 'United Kingdom': 'Europe',
 'United Korea (former)\r\n': 'Asia',
 'United States': 'America',
 'Venezuela': 'America',
 'Vietnam': 'Asia',
 'Virgin Islands, British': 'America',
 'Wake Island': 'Oceania',
 'Wallis et Futuna': 'Oceania',
 'West Bank and Gaza': 'Europe',
 'West Germany': 'Europe',
 'Yugoslavia': 'Europe',
 '\x81land': 'Nan', 
 'Central African Rep.': 'Africa',
 'Czech Rep.': 'Europe',
 'Korea, Dem. Rep.': 'Asia',
 'Dominican Rep.': 'America',
 'Laos': 'Asia',
 'Korea, Rep.': 'Asia',
 'Yemen, Rep.': 'Asia'}

cleaned_year_data.Continent = cleaned_year_data.Continent.fillna(cleaned_year_data.Country.map(missing_conti_map))
child_death.Continent = child_death.Continent.fillna(child_death.Country.map(missing_conti_map))

<a id='eda'></a>
## Exploratory Data Analysis



### Research Question 1 (Replace this header name!)

In [ ]:
child_death_top10_1 = child_death[['Country', 'Total']].sort_values(['Total'], ascending = False).iloc[0:9]
child_death_top10_1.plot('Country', 'Total', kind='bar')
plt.title('Bar of Total Death in 2008')
plt.show()

In [ ]:
child_death.head()

In [ ]:
child_death_top10_2 = child_death[['Country', 'Num_per_thousand']].sort_values('Num_per_thousand', ascending=False).iloc[:9]
child_death_top10_2.plot('Country', 'Num_per_thousand', kind = 'bar')

In [ ]:
coi = list(set(list(child_death_top10_2.Country) + list(child_death_top10_1.Country)))

sub = ttl_year_data.loc[ttl_year_data.Country.isin(coi)]

sub.head(5)

In [ ]:
for country, group in sub.groupby('Country'):
    plt.plot(group['Year'], group['Est_HIV_pct_15-49'], label=country)
# plt.legend()

plt.legend()
plt.show()

### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!